# Post-processing of the semi-automated selection

This notebook is used to load the results after the phase 1 of the selection.

This is in preparation for the phase 2 of the selection (selection by hand).

We do the following:
1. Loading the results of the 3 data sources
2. Merging these results
3. Removing all dupplicates (within and in between data sources)
4. Create the excel file for the phase 2 of the selection (selection by hand)

***

**Importing libraries:**

In [1]:
import pandas as pd
import os
import json
import re
from pprint import pprint
from collections import Counter
#!pip install xlsxwriter
import xlsxwriter

## 1. Loading results

### 1.1. Load results from IEEE

In [2]:
_path_ieee_keep = os.path.join('..', '2_automatic_selection', '1_results_of_exclusion', 'ieee_df_20230718-173758.csv')

cols= ['original_title', 'PDF Link', 'Abstract', 'Authors', 'Author Keywords',
       'Publication Year','ISBNs', 'DOI', 'Publication Title', 'title', 'keep_title']
df_ieee = pd.DataFrame(pd.read_csv(_path_ieee_keep, usecols=cols))

df_ieee = df_ieee.rename(columns={'ISBNs': 'isbn'})
df_ieee = df_ieee.rename(columns={'DOI': 'doi'})
df_ieee = df_ieee.rename(columns={'PDF Link': 'link'})
df_ieee = df_ieee.rename(columns={'Abstract': 'abstract'})
df_ieee = df_ieee.rename(columns={'Publication Title': 'pub_info'})
df_ieee = df_ieee.rename(columns={'Publication Year': 'date'})
df_ieee = df_ieee.rename(columns={'PDF Link': 'link'})
df_ieee = df_ieee.rename(columns={'Publication Title': 'pub_info'})
df_ieee = df_ieee.rename(columns={'Author Keywords': 'keywords'})
df_ieee = df_ieee.rename(columns={'Authors': 'authors'})
df_ieee = df_ieee.rename(columns={'title': 'title_lower'})
df_ieee = df_ieee.rename(columns={'original_title': 'title'})

df_ieee = pd.concat([df_ieee.copy(deep=True),
                     pd.DataFrame(['ieee']*df_ieee.shape[0], columns=['data_source']),
                     pd.DataFrame([None]*df_ieee.shape[0], columns=['article']),
                     pd.DataFrame([None]*df_ieee.shape[0], columns=['journal_or_book']),
                     pd.DataFrame([None]*df_ieee.shape[0], columns=['publisher']),
                     pd.DataFrame([1]*df_ieee.shape[0], columns=['bib']),
                     pd.DataFrame([None]*df_ieee.shape[0], columns=['issn']),
                     pd.DataFrame([None]*df_ieee.shape[0], columns=['result_id']),
                     pd.DataFrame([None]*df_ieee.shape[0], columns=['count'])
                     ], axis=1)

cols= ['title', 'link', 'abstract', 'authors', 'keywords', 'date','isbn', 
       'doi', 'pub_info', 'title_lower', 'keep_title', 'bib', 'result_id', 'article', 
       'publisher', 'journal_or_book', 'count', 'data_source']


df_ieee = df_ieee[cols]
print('Shape of df_ieee: ', df_ieee.shape)
print('Columns of df_ieee: ', df_ieee.columns.values)

Shape of df_ieee:  (421, 18)
Columns of df_ieee:  ['title' 'link' 'abstract' 'authors' 'keywords' 'date' 'isbn' 'doi'
 'pub_info' 'title_lower' 'keep_title' 'bib' 'result_id' 'article'
 'publisher' 'journal_or_book' 'count' 'data_source']


### 1.2. Load results from ACM

In [3]:
_path_acm = os.path.join('..', '2_automatic_selection', '1_results_of_exclusion','acm_df_20230718-172831.csv')
cols = ['original_title', 'ISBN', 'ISSN', 'DOI', 'URLs',
       'Proceedings title', 'Abstract', 'Authors', 'Journal',
       'Publication year', 'Date published', 'URLs.1', 'title', 'keep_title']
df_acm = pd.DataFrame(pd.read_csv(_path_acm, usecols = cols))

l1 = df_acm['Proceedings title'].values
l1 = [e if e==e else "" for e in l1]
l2 = df_acm['Journal'].values
l2 = [e if e==e else "" for e in l2]
res = list(map(lambda x: x[0] + x[1], zip(l1, l2)))

my_columns = ['Title', 'ISBN', 'ISSN', 'DOI', 'URLs', 'Proceedings title', 'Abstract', 'Authors', \
              'Journal', 'Publication year', 'Date published', 'URLs']

df_acm = df_acm.rename(columns={'title': 'title_lower'})
df_acm = df_acm.rename(columns={'original_title': 'title'})
df_acm = df_acm.rename(columns={'ISBN': 'isbn'})
df_acm = df_acm.rename(columns={'DOI': 'doi'})
df_acm = df_acm.rename(columns={'ISSN': 'issn'})
df_acm = df_acm.rename(columns={'Abstract': 'abstract'})
df_acm = df_acm.rename(columns={'Authors': 'authors'})
df_acm = df_acm.rename(columns={'Publication year': 'date'})
df_acm = df_acm.rename(columns={'URLs': 'link'})

df_acm = pd.concat([df_acm.copy(deep=True), 
                     pd.DataFrame(['acm']*df_acm.shape[0], columns=['data_source']),
                     pd.DataFrame(res, columns=['pub_info']),
                     pd.DataFrame([None]*df_acm.shape[0], columns=['article']),
                     pd.DataFrame([0]*df_acm.shape[0], columns=['keywords']),
                     pd.DataFrame([None]*df_acm.shape[0], columns=['journal_or_book']),
                     pd.DataFrame([None]*df_acm.shape[0], columns=['publisher']),
                     pd.DataFrame([1]*df_acm.shape[0], columns=['bib']),
                     pd.DataFrame([None]*df_acm.shape[0], columns=['result_id']),
                     pd.DataFrame([None]*df_acm.shape[0], columns=['count'])
                     ], axis=1)

cols= ['title', 'link', 'abstract', 'authors', 'keywords', 'date','isbn', 
       'doi', 'pub_info', 'title_lower', 'keep_title', 'bib', 'result_id', 'article', 
       'publisher', 'journal_or_book', 'count', 'data_source']

df_acm = df_acm[cols]

print('Shape of df_acm: ', df_acm.shape)
print('Columns of df_acm: ', df_acm.columns.values)

Shape of df_acm:  (137, 18)
Columns of df_acm:  ['title' 'link' 'abstract' 'authors' 'keywords' 'date' 'isbn' 'doi'
 'pub_info' 'title_lower' 'keep_title' 'bib' 'result_id' 'article'
 'publisher' 'journal_or_book' 'count' 'data_source']


### 1.3. Load results from Scholar

**Load organic results:**

In [4]:
_path_scholar_keep = os.path.join('..', '2_automatic_selection', '1_results_of_exclusion', 'scholar_keep_20230711-120831.csv')
cols = ['original_title', 'result_id', 'publication_info_summary', 'link', 'title', 'keep_title', 'count']
df = pd.DataFrame(pd.read_csv(_path_scholar_keep, usecols=cols))

df = df.rename(columns={'title': 'title_lower'})
df = df.rename(columns={'original_title': 'title'})
df = df.rename(columns={'publication_info_summary': 'pub_info'})

df = pd.concat([df.copy(deep=True),
                pd.DataFrame(['scholar']*df.shape[0], columns=['data_source']),
                pd.DataFrame([None]*df.shape[0], columns=['abstract']),
                pd.DataFrame([None]*df.shape[0], columns=['keywords']),
                pd.DataFrame([None]*df.shape[0], columns=['issn']),
                pd.DataFrame([None]*df.shape[0], columns=['doi']),
                pd.DataFrame([None]*df.shape[0], columns=['isbn']),
                pd.DataFrame([0]*df.shape[0], columns=['bib']),
                pd.DataFrame([None]*df.shape[0], columns=['article']),
                pd.DataFrame([None]*df.shape[0], columns=['authors']),
                pd.DataFrame([None]*df.shape[0], columns=['date']),
                pd.DataFrame([None]*df.shape[0], columns=['journal_or_book']),
                pd.DataFrame([None]*df.shape[0], columns=['publisher'])
                ], axis=1)

cols = ['title', 'link', 'abstract', 'authors', 'keywords', 'date', 'isbn',
       'doi', 'pub_info', 'title_lower', 'keep_title', 'bib', 'result_id', 'article',
       'publisher', 'journal_or_book', 'count', 'data_source']

#set_difference = set(df.columns.tolist()) - set(cols)
#any (e not in df.columns.tolist() for e in cols)
#any (e not in cols for e in df.columns.tolist())

df = df[cols]

print('Shape of df: ', df.shape)
print('Columns of df: ', df.columns.values)

Shape of df:  (2589, 18)
Columns of df:  ['title' 'link' 'abstract' 'authors' 'keywords' 'date' 'isbn' 'doi'
 'pub_info' 'title_lower' 'keep_title' 'bib' 'result_id' 'article'
 'publisher' 'journal_or_book' 'count' 'data_source']


**Load citation results:**

In [5]:
# citation results from serpAPI:
_path_a = os.path.join('serpapi_cite_results','20230717-165430_row_1004.json')
_path_b = os.path.join('serpapi_cite_results','20230717-173402_row_1635.json')
_path_c = os.path.join('serpapi_cite_results','20230718-093910_row_2588.json')
with open(_path_a, 'r') as f:
    all_results_a = json.load(f)
with open(_path_b, 'r') as f:
    all_results_b = json.load(f)
with open(_path_c, 'r') as f:
    all_results_c = json.load(f)
all_results_2 = all_results_a | all_results_b | all_results_c
print(len(all_results_2))

2589


In [6]:
def get_fields_3(reference):
    """Function to recover bibliographic data"""

    full_reference = reference
    first_part = ""
    found_authors = False
    while not found_authors:
        dot = reference.find('.')
        before_dot = reference[:dot]
        first_part += before_dot+'.'

        if len(before_dot.split(" ")[-1]) != 1 or reference[dot:dot+3]=='. "':
            found_authors = True
            authors = first_part

        reference = reference[dot +1 :]

    title = re.search(r"\"(.+?)\"", reference)
    if title:
        title = title.group(1)
        reference = reference.replace(" \""+title+"\" ", "")

    date = re.search(r".*(\([1-3][0-9]{3}\))", reference)
    if date:
        date = date.group(1)
        pos_date = reference.find(date)
        reference = reference.replace(" "+date, "")
        date = date[1:-1]

    dot = reference.find('.')

    first_part = ""
    found_journal_or_book = False
    while not found_journal_or_book:
        dot = reference.find('.')
        before_dot = reference[:dot]
        first_part += before_dot+'.'
        if before_dot.split(" ")[-1] != "pp" and before_dot.split(" ")[-1] != "no":
            found_journal_or_book = True
            journal_or_book = first_part

        reference = reference[dot +2 :]
    
    if not date:
        date = re.search(r".*((?<!\d)[1-3][0-9]{3}(?!\d))", reference)
        if date:
            date = date.group(1)
            date_pos = reference.find(date)
            if reference[date_pos-2:date_pos] == ", ":
                reference = reference.replace(", "+date+".", "")
            else:
                reference = reference.replace(date+".", "")

    publisher = reference

    title=title if title != None else ''
    authors=authors if authors != None else ''
    date=date if date != None else ''
    journal_or_book=journal_or_book if journal_or_book != None else ''
    publisher=publisher if publisher != None else ''
    
    res_dict = {
        'article' : ''.join(title),
        'authors': ''.join(authors),
        'date' : date,
        'journal_or_book' : ''.join(journal_or_book),
        'publisher' : ''.join(publisher)
    }
    
    return res_dict

In [7]:
cpt=0
no_links=0
no_citations=0
no_chicago=0

for result_id in all_results_2.keys():
    #print("x --------------- x")
    #print(f"      {cpt}         ")
    #print("x                 x")

    results = all_results_2[result_id]
    if "links" in results.keys():
        bibtex = results["links"][0]["link"]
    else:
        no_links +=1
    
    if "citations" in results.keys():
        citations = {item['title']: item['snippet'] for item in results['citations']}

        if "Chicago" in citations.keys():
            reference = citations['Chicago']
            res_dict = get_fields_3(reference)
            
            for key in res_dict.keys():
                df.loc[df['result_id']==result_id, key] = res_dict[key] 
                #df.loc[df['A'] > 2, 'B'] = new_val
            df.loc[df['result_id']==result_id, 'bib'] = 1
        else:
            no_chicago +=1
    else:
        no_citations +=1
    
    cpt+=1
print(no_citations, no_links, no_chicago)

69 69 22


**Studies without any citation data:**

In [8]:
for i, row in df[df['bib']==0].iterrows():
    print("x ---------- x")
    print(row['title'])
    print(row['pub_info'])

x ---------- x
Estimation of Remaining Useful Lifetime of Power Electronic Components with Machine Learning based on Mission Profile Data
D Bhat, S Muench, M Roellig - Power Electronic Devices and Components, 2023 - Elsevier
x ---------- x
Energy Consumption Estimation And Management With Machine Learning Regression Techniques
AG TOPRAK, S ŞAHİN, AY MUTLU - researchgate.net
x ---------- x
Genetic algorithm based optimized feature engineering and hybrid machine learning for effective energy consumption prediction
PW Khan, YC Byun - IEEE Access, 2020 - ieeexplore.ieee.org
x ---------- x
… nonlinear displacement ratio prediction of self-centering energy-absorbing dual rocking core system under near-fault ground motions using machine learning
S Hu, W Wang, MS Alam - Journal of Earthquake Engineering, 2021 - Taylor & Francis
x ---------- x
Modeling Energy Consumption Using Machine Learning
SA Sarswatula, T Pugh, V Prabhu - Frontiers in Manufacturing …, 2022 - frontiersin.org
x ---------- x


In [9]:
df[df['bib']==0].to_csv('no_citations_results-selection-by_hand.csv')

## 2. Concatenate results of 3 datasource

In [10]:
df_all = pd.concat([df, df_ieee, df_acm], axis=0, ignore_index=True)


df_all = pd.concat([df_all.copy(deep=True),
                   pd.DataFrame([None]*df_all.shape[0], columns=['keep_duplicate']),
                   ], axis=1)

df_all.shape

(3147, 19)

## 3. Treating duplicates

### 3.1. Duplicates wihtin Scholar

In [11]:
# Find the duplicates

titles = [title for title in df_all['title_lower'].tolist()]
titles_nb = len(titles)
print("Number of titles: ", titles_nb)

titles_dict = dict(Counter(titles))
title_counts = []
for i, row in df_all.iterrows():
    title_counts.append(titles_dict[row['title_lower']])

for i in range(len(title_counts)):
    df_all.loc[i, 'count'] = title_counts[i]

dupl_df = df_all[df_all['count'] > 1]
nodupl_df = df_all[df_all['count'] == 1]

dupl_set = set(dupl_df['title'].to_list())

nodupl_df.shape[0]

Number of titles:  3147


2091

We create a csv to check **manually** which duplicates are to be kept.

In [12]:
for i, row in df_all.iterrows():
    if row['count'] > 1:
        df_tmp = df_all.loc[df_all['title_lower'] == row['title_lower']]
        #display(df_tmp[['title', 'link', 'authors', 'date', 'pub_info']])  
        
# about scholar duplicates
df_check_dupl = pd.DataFrame()
indexes = []
for i, row in df.iterrows(): # create a csv to check which duplicates are to be kept by hand
    if row['count'] > 1:
        df_tmp = df.loc[df['title_lower'] == row['title_lower']]
        new_index = df_tmp.index.values
        if new_index[0] not in indexes:
            indexes.extend(new_index)
            df_check_dupl = pd.concat([df_check_dupl.copy(deep=True), df_tmp[['title', 'link', 'authors', 'date', 'pub_info', 'result_id']]], axis=0)
#df_check_dupl.to_csv("test.csv") # for saving

We load the modified csv, it has the additional column 'keep_dupl'.

In [13]:
# loading the modified csv, it has the additional column 'keep_dupl'
df_scholar_dupl = pd.DataFrame(pd.read_csv('check_dupl_2.csv', encoding = 'unicode_escape', sep = ';'))

# removing the Scholar duplicates: we add the column keep duplicates to df_all and fill it with the right values for Scholar:
for i, row in df_scholar_dupl.iterrows():
    df_all.loc[df_all['result_id'] == row['result_id'], 'keep_duplicate'] = row['keep_dupl']

### 3.2. Treat all other duplicates

In [14]:
# Find the duplicates:
titles = [title for title in df_all['title_lower'].tolist()]
titles_nb = len(titles)
print("Number of titles: ", titles_nb)

titles_dict = dict(Counter(titles))
title_counts = []

for i, row in df_all.iterrows():
    title_counts.append(titles_dict[row['title_lower']])

for i in range(len(title_counts)):
    df_all.loc[i, 'count'] = title_counts[i]

dupl_df = df_all[df_all['count'] > 1]
nodupl_df = df_all[df_all['count'] == 1]

dupl_set = set(dupl_df['title'].to_list())

Number of titles:  3147


In [15]:
# Treat all other duplicates:

df_check_dupl = pd.DataFrame()

warning = 0

indexes = []
for i, row in df_all.iterrows():
    
    if row['count'] > 1: # there is a dupplicate
        
        
        df_tmp = df_all.loc[df_all['title_lower'] == row['title_lower'], :] # rows with this duplicate title
        new_index = df_tmp.index.values                                     # .. and their indexes
        
        if new_index[0] not in indexes:                   # has not already been treated            
            sources = df_tmp['data_source'].tolist()      # data sources of these duplicates          
            keep_list = df_tmp['keep_duplicate'].tolist() # previous values of keep_duplicate column
            
            
            if len(set(sources))==1:
                if sources[0]=='acm':                     # we keep the first one
                    keep_list = [0]*df_tmp.shape[0]; keep_list[0]=1
#                 if sources[0]=='scholar':               # in the case of scholar we do nothing
#                     pass
#                 if sources[0] == 'ieee':                # does not happen
        
    
            elif 'scholar' not in sources:                # we keep ieee (there is never just ieee and acm)
                keep_list = [1 if sources[kk]=='ieee' else 0 for kk in range(len(sources))]
                
            elif 'scholar' in sources:                    
                count_data_sources = dict(Counter(sources))
                if count_data_sources['scholar'] > 1:     # several scholar -> we already decided which one to keep
                    for ll in range(len(keep_list)):      # put everything else to zero
                        if keep_list[ll] == None:
                            keep_list[ll] = 0
                else:                                     # keep the scholar citation
                    keep_list = [1 if sources[kk]=='scholar' else 0 for kk in range(len(sources))]
            
            # UNCOMMENT TO DISPLAY MODIFICATIONS:
            # print("x ------------------------ x")
            # display(df_tmp[['title', 'link', 'authors', 'date', 'pub_info','data_source', 'result_id', 'keep_duplicate']])
            # print(df_all.loc[df_all['title_lower'] == row['title_lower'], 'keep_duplicate'])
            df_all.loc[df_all['title_lower'] == row['title_lower'], 'keep_duplicate'] = keep_list
            # print(df_all.loc[df_all['title_lower'] == row['title_lower'], 'keep_duplicate'])
            
            indexes.extend(new_index)
            df_check_dupl = pd.concat([df_check_dupl.copy(deep=True), df_tmp[['title', 'link', 'authors', 'date', 'pub_info','data_source', 'result_id', 'keep_duplicate']]], axis=0)

print('warning', warning)
# df_check_dupl.to_csv("check_dupl_all.csv") # for saving

warning 0


In [16]:
print(df_all[df_all['count']>1].shape[0])
print(df_all[df_all['count']==2].shape[0]+df_all[df_all['count']==3].shape[0]+df_all[df_all['count']==4].shape[0])
print(df_all[df_all['keep_duplicate']==0].shape[0] + df_all[df_all['keep_duplicate']==1].shape[0])

1056
1056
1056


### 3.3. Remove the duplicates

In [17]:
df_no_dupl = df_all[df_all['keep_duplicate'] != 0]
#df_no_dupl = df_all.drop(df_all[df_all.keep_duplicate != 0].index)
df_no_dupl = df_no_dupl.reset_index(drop=True)
df_no_dupl = pd.concat([pd.DataFrame([str(i+1) for i in range(df_no_dupl.shape[0])], columns=['id']), df_no_dupl.copy(deep=True)], axis=1)

In [18]:
df_no_dupl.shape

(2607, 20)

In [19]:
for i, row in df_no_dupl.iterrows():
    if row['data_source'] == 'scholar':
        if row['article']:
            tmp = row['authors']+row['journal_or_book']+row['publisher']
            df_no_dupl.loc[i, 'pub_info'] = tmp
        elif row['authors']:
            tmp = row['authors']+row['publisher']
            df_no_dupl.loc[i, 'pub_info'] = tmp

## 4. Save as an Excel file

In [20]:
def create_excel(start, stop, file_name):

    # Create a sample dataframe

    df_save = df_no_dupl.loc[start-1:stop-1, ['id', 'title', 'link', 'date', 'pub_info',
                          'data_source']]

    df_save = pd.concat([df_save.copy(deep=True),
                       pd.DataFrame([""]*df_save.shape[0], columns=['keep']),
                       pd.DataFrame([""]*df_save.shape[0], columns=['don_t_know']),
                       pd.DataFrame([""]*df_save.shape[0], columns=['discard']),
                       pd.DataFrame([""]*df_save.shape[0], columns=['comments_from_reviewer']),
                       ], axis=1)


    # Apply custom colors to the dataframe
    color_mapping = {
        'keep': 'green',
        'don_t_know': 'orange',
        'discard': 'red',
        'comments_from_reviewer': 'yellow'
    }

    # Create an Excel workbook and worksheet
    workbook = xlsxwriter.Workbook(file_name)
    worksheet = workbook.add_worksheet()

    # Write the headers
    headers = list(df_save.columns)
    for col_num, header in enumerate(headers):
        worksheet.write(0, col_num, header)

    # Write the data and apply background color
    for row_num, row_data in enumerate(df_save.values):
        for col_num, cell_data in enumerate(row_data):
            cell_data = cell_data if cell_data == cell_data else ""
            worksheet.write(row_num + 1, col_num, cell_data)
            #if df.columns[col_num] == 'City' and cell_data in color_mapping:
            #    cell_format = workbook.add_format({'bg_color': color_mapping[cell_data]})
            #    worksheet.write(row_num + 1, col_num, cell_data, cell_format)
            if df_save.columns[col_num] == 'keep':
                cell_format = workbook.add_format({'bg_color': color_mapping['keep']})
                worksheet.write(row_num + 1, col_num, cell_data, cell_format)
            if df_save.columns[col_num] == 'don_t_know':
                cell_format = workbook.add_format({'bg_color': color_mapping['don_t_know']})
                worksheet.write(row_num + 1, col_num, cell_data, cell_format)
            if df_save.columns[col_num] == 'discard':
                cell_format = workbook.add_format({'bg_color': color_mapping['discard']})
                worksheet.write(row_num + 1, col_num, cell_data, cell_format)
            if df_save.columns[col_num] == 'comments_from_reviewer':
                cell_format = workbook.add_format({'bg_color': color_mapping['comments_from_reviewer']})
                worksheet.write(row_num + 1, col_num, cell_data, cell_format)

    # Save the Excel file
    workbook.close()

# create_excel(0, 2660, 'test.xlsx')